In [27]:
import pandas as pd
#from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras import Sequential
from keras.layers import Dense, Dropout
import keras.metrics
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, TensorBoard
from MLscores import calc_metrics, metrics_dict, cmvals, recall, hybridrecall
import keras.backend as K
import tensorflow as tf
from functools import partial
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import recall_score, make_scorer
from xgboost import XGBClassifier
from check_and_prepare_dataset import load_dataset, prepare_dataset
from manage_model import create_NN_model, create_sklearn_model, allowgrowthgpus
#import eli5
#from eli5.sklearn import PermutationImportance

In [2]:
#params={'ES_mindelta': 0.001, 'ES_monitor': 'loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 10}, 'dropout': 0.1, 'feature_drop': ('dir_max', 'dom_dir', 'month', 'wkd'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 70}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
#params={'ES_mindelta': 0.0001, 'ES_monitor': 'loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': 0.3, 'feature_drop': ('dir_max', 'dom_dir', 'month', 'wkd'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 200.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params = {}
params['nn_auc']= {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_h1'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 2}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 60.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_h2'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_nh2'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 1700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_nh5'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 10}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 1100.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}

In [44]:
#X, y, g=load_dataset('/home/aapostolakis/Documents/ffpdata/newcrossval/datasets/randomnofire/old_random_new_features_norm.csv',\
#                    featuredrop=params['feature_drop'])
X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv',['dir_max','dom_dir','weekday','month'])
                      #featuredrop=['corine','dir_max','dom_dir','wkd','month'])

Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_7', 'bin_month_5', 'bin_dir_max_3', 'bin_weekday_4', 'bin_weekday_2', 'bin_dom_dir_7', 'bin_dir_max_4', 'bin_dir_max_6', 'bin_dir_max_8', 'bin_dom_dir_8', 'bin_dir_max_2', 'bin_dom_dir_2', 'bin_month_10', 'bin_dom_dir_6', 'bin_dom_dir_4', 'bin_dom_dir_3', 'bin_month_9', 'bin_dom_dir_5', 'bin_month_8', 'bin_month_4', 'bin_dir_max_1', 'bin_dom_dir_1', 'bin_month_3', 'bin_weekday_7', 'bin_month_6', 'bin_weekday_1', 'bin_weekday_6', 'bin_month_7', 'bin_weekday_5', 'bin_weekday_3', 'bin_dir_max_5']


In [45]:
X

,bin_corine_14,bin_corine_gr1,bin_corine_44,bin_corine_7,bin_corine_18,bin_corine_41,bin_corine_gr32,max_temp,dom_vel,bin_corine_5,...,bin_corine_gr31,bin_corine_6,lst_day,frequency,bin_corine_gr22,dem,pop,bin_corine_31,bin_corine_gr5,f81
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.568711,0.430780,0.0,...,0.000000,0.0,0.62528,0.0,0.330564,0.007330,0.001916,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.592992,0.621212,0.090392,0.0,...,0.407008,0.0,0.52640,0.0,0.000000,0.494835,0.001959,0.0,0.0,0.011601
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.654663,0.050841,0.0,...,0.684398,0.0,0.59536,0.0,0.315602,0.111508,0.007423,0.0,0.0,0.819954
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.599968,0.173459,0.0,...,0.000000,0.0,0.60168,0.0,0.000000,0.022838,0.040576,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.726821,0.167867,0.0,...,0.000000,0.0,0.62136,0.0,0.793981,0.016968,0.008994,0.0,0.0,0.008353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41679,0.0,0.0,0.0,0.0,0.0,0.0,0.119075,0.651412,0.071011,0.0,...,0.663003,0.0,0.62232,0.0,0.000000,0.098280,0.006118,0.0,0.0,0.094664
41680,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.651412,0.071011,0.0,...,0.990445,0.0,0.62176,0.0,0.000000,0.093271,0.005807,0.0,0.0,0.094664
41681,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.651412,0.071011,0.0,...,1.000000,0.0,0.63968,0.0,0.000000,0.080038,0.005674,0.0,0.0,0.094200
41682,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.651412,0.071011,0.0,...,0.022333,0.0,0.62232,0.0,0.000000,0.095567,0.006080,0.0,0.0,0.094664


In [4]:
class MakeModel(object):

    def __init__(self, X=None, y=None):
        pass

    def predict(self, X):
        y_pred = self.model.predict(X)
        return y_pred
        #return np.argmax(y_pred, axis=1)
    
    def fit(self, X, y):
        skwrapped_model = KerasClassifier(build_fn=creatennmodel,
                                          train_input=X,
                                          epochs=2,
                                          batch_size=512,
                                          #validation_split=1-TRAIN_TEST_SPLIT,
                                          verbose=0)
        self.model = skwrapped_model
        self.model.fit(X, y)
        return self.model

In [5]:
models={}
for paramset in params:
    creatennmodel = partial(create_NN_model, params[paramset], X)
    models[paramset] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)

/tmp/ipykernel_3222130/137153893.py:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)


In [6]:
paramssk={}
#RF best
paramssk['RF']={'algo': 'RF','bootstrap': True, 'class_weights': {0: 1, 1: 10}, 'criterion': 'entropy', 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_depth': 1000, 'max_features': 2.0, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 350}
#ET best
paramssk['ET']={'algo': 'XT', 'bootstrap': True, 'class_weights': {0: 1, 1: 1}, 'criterion': 'entropy', 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_depth': None, 'max_features': 2.0, 'min_samples_leaf': 50, 'min_samples_split': 150, 'n_estimators': 800}
#XG
paramssk['XGB']={'algo': 'XGB', 'alpha': 60, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'gamma': 0.1, 'lambda': 21.0, 'max_depth': 44.0, 'n_estimators': 640, 'scale_pos_weight': 15, 'subsample': 0.6}
for skalgo in paramssk:
    print(paramssk[skalgo]['algo'])
    models[skalgo] = create_sklearn_model(paramssk[skalgo],X)

RF
XT
XGB


In [15]:
models

{'nn_auc': <keras.wrappers.scikit_learn.KerasClassifier at 0x7f537246baf0>,
 'nn_h1': <keras.wrappers.scikit_learn.KerasClassifier at 0x7f537246afe0>,
 'nn_h2': <keras.wrappers.scikit_learn.KerasClassifier at 0x7f537246bd00>,
 'nn_nh2': <keras.wrappers.scikit_learn.KerasClassifier at 0x7f53724acbe0>,
 'nn_nh5': <keras.wrappers.scikit_learn.KerasClassifier at 0x7f53724afdf0>,
 'RF': RandomForestClassifier(class_weight={0: 1, 1: 10}, criterion='entropy',
                        max_depth=1000, max_features=21, min_samples_leaf=10,
                        n_estimators=350, n_jobs=8),
 'ET': ExtraTreesClassifier(bootstrap=True, class_weight={0: 1, 1: 1},
                      criterion='entropy', max_features=21, min_samples_leaf=50,
                      min_samples_split=150, n_estimators=800, n_jobs=8),
 'XGB': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_ro

In [46]:
#models1={'RF':models['RF'], 'ET':models['ET']}
allowgrowthgpus()
for modname in models:
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv',params['feature_drop'])
    print(modname)
    models[modname].fit(X,y)
    break

1 Physical GPUs, 1 Logical GPUs


KeyError: 'feature_drop'

In [8]:
results_lists={}
for modname in models:
    print(modname)
    #models[modname].fit(X,y)
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv',params['feature_drop'])
    r=permutation_importance(models[modname], X, y, n_repeats=3,random_state=0, scoring='roc_auc')
    results_lists[modname]=[]
    for i in r.importances_mean.argsort()[::-1]:
        #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        if not any(c in X.columns[i] for c in ['dir_max','dom_dir','wkd','month']):
            results_dict={}
            results_dict['rank '+modname]=X.columns[i]
            results_dict['perm. imp.'+modname]=r.importances_mean[i]
            results_lists[modname]+=[results_dict]

nn_auc
1303/1303 [==============================] - 2s 1ms/step


1303/1303 [==============================] - 2s 1ms/step


1303/1303 [==============================] - 1s 798us/step
nn_h1
1303/1303 [==============================] - 2s 1ms/step


1303/1303 [==============================] - 1s 1ms/step


1303/1303 [==============================] - 1s 1ms/step
nn_h2
1303/1303 [==============================] - 2s 1ms/step


1303/1303 [==============================] - 2s 1ms/step


1303/1303 [==============================] - 1s 1ms/step


KeyboardInterrupt: 

In [40]:
X.columns

Index(['bin_corine_14', 'bin_corine_gr1', 'bin_corine_44', 'bin_corine_7',
       'bin_corine_18', 'bin_corine_41', 'bin_corine_gr32', 'max_temp',
       'dom_vel', 'bin_corine_5',
       ...
       'bin_month_8', 'bin_month_9', 'bin_month_10', 'bin_weekday_1',
       'bin_weekday_2', 'bin_weekday_3', 'bin_weekday_4', 'bin_weekday_5',
       'bin_weekday_6', 'bin_weekday_7'],
      dtype='object', length=105)

In [21]:
recall_score(y,ypred)

0.9983417447728912

In [34]:
rec1_scorer = make_scorer(recall_score, pos_label=1)
rec0_scorer = make_scorer(recall_score, pos_label=0)
results_lists={}
scoring={'recall1':rec1_scorer, 'recall0':rec0_scorer}
for modname in models:
    print(modname)
    #models[modname].fit(X,y)
    #ypred=models['nn_auc'].predict(X)
    r=permutation_importance(models[modname], X, y, n_repeats=1, scoring=scoring)
    for s in scoring:
        results_lists['%s %s'%(modname,s)]=[]
        for i in r[s].importances_mean.argsort()[::-1]:
            #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
            if not any(c in X.columns[i] for c in ['dir_max','dom_dir','wkd','month']):
                results_dict={}
                results_dict['rank %s %s'%(modname,s)]=X.columns[i]
                results_dict['perm. imp. %s %s'%(modname,s)]=r[s].importances_mean[i]
                results_lists['%s %s'%(modname,s)]+=[results_dict]
    break

nn_auc
1303/1303 [==============================] - 1s 1ms/step


In [36]:
results_lists

{'nn_auc recall1': [{'rank nn_auc recall1': 'f81',
   'perm. imp. nn_auc recall1': 0.1323720259552993},
  {'rank nn_auc recall1': 'res_max',
   'perm. imp. nn_auc recall1': 0.1080749819754867},
  {'rank nn_auc recall1': 'bin_weekday_3',
   'perm. imp. nn_auc recall1': 0.09437635183850035},
  {'rank nn_auc recall1': 'bin_weekday_7',
   'perm. imp. nn_auc recall1': 0.08752703677000728},
  {'rank nn_auc recall1': 'bin_corine_gr32',
   'perm. imp. nn_auc recall1': 0.08327325162220622},
  {'rank nn_auc recall1': 'bin_weekday_6',
   'perm. imp. nn_auc recall1': 0.0741888968997837},
  {'rank nn_auc recall1': 'bin_weekday_2',
   'perm. imp. nn_auc recall1': 0.07253064167267487},
  {'rank nn_auc recall1': 'bin_weekday_5',
   'perm. imp. nn_auc recall1': 0.07123287671232881},
  {'rank nn_auc recall1': 'bin_weekday_4',
   'perm. imp. nn_auc recall1': 0.07101658255227117},
  {'rank nn_auc recall1': 'bin_weekday_1',
   'perm. imp. nn_auc recall1': 0.06409516943042537},
  {'rank nn_auc recall1': 'xp

In [14]:
#r=permutation_importance(sknnmodel, X, y, n_repeats=3,random_state=0)

In [15]:
#for i in r.importances_mean.argsort()[::-1]:
    #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#    print(f"{X.columns[i]:<17}"
#        f"{r.importances_mean[i]:.3f}"
#        f" +/- {r.importances_std[i]:.3f}")

In [37]:
results_pd=[]
for mod in results_lists:
    results_pd+=[pd.DataFrame(results_lists[mod])]
all_res_pd=pd.concat(results_pd, axis=1)

In [43]:
all_res_pd.iloc[:20]

,rank nn_auc recall1,perm. imp. nn_auc recall1,rank nn_auc recall0,perm. imp. nn_auc recall0
0,f81,0.132372,bin_weekday_4,0.022507
1,res_max,0.108075,bin_weekday_2,0.022255
2,bin_weekday_3,0.094376,bin_corine_gr32,0.022111
3,bin_weekday_7,0.087527,bin_weekday_7,0.021464
4,bin_corine_gr32,0.083273,bin_weekday_1,0.021248
5,bin_weekday_6,0.074189,bin_weekday_3,0.020637
6,bin_weekday_2,0.072531,bin_weekday_6,0.018768
7,bin_weekday_5,0.071233,bin_weekday_5,0.018228
8,bin_weekday_4,0.071017,res_max,0.018084
9,bin_weekday_1,0.064095,xpos,0.013662


In [17]:
all_res_pd.to_csv('results/permimpall.csv', index=False)